In [1]:
import sys
from dotenv import load_dotenv
import os

load_dotenv()

PROJECT_ROOT = os.getenv("PROJECT_ROOT")
sys.path.append(PROJECT_ROOT)
sys.path.append(os.path.join(PROJECT_ROOT, "src"))
sys.path.append(os.path.join(PROJECT_ROOT, "src/scripts"))
sys.path.append(os.path.join(PROJECT_ROOT, "src/utils"))

# Now import the module
from model_lora_finetune import *

/home/kotsios/dsit/deep_learning/resume_to_json/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 09-19 21:40:09 [__init__.py:216] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.5: Fast Gemma3_Text patching. Transformers: 4.56.1. vLLM: 0.10.2.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 1. Max memory: 23.663 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Making `model.base_model.model.model` require gradients


Filter: 100%|██████████| 100/100 [00:00<00:00, 446.68 examples/s]


In [ ]:
args = TrainingArguments(
	per_device_train_batch_size=12,
	gradient_accumulation_steps=24,
	warmup_ratio=0.05,
	num_train_epochs=3,
	learning_rate=2e-4,
	fp16=not is_bfloat16_supported(),
	bf16=is_bfloat16_supported(),
	logging_steps=10,
	optim="adamw_torch",
	weight_decay=0.01,
	lr_scheduler_type="linear",
	seed=3407,
	output_dir=os.path.join(MODELS_PATH, "lora_finetuned_gemma-3-1b-it-4bit"),
	report_to="tensorboard",
	eval_strategy="steps",
	eval_steps=10,
	save_strategy="steps",
	save_steps=10,
	save_total_limit=3,
	metric_for_best_model="eval_loss",
	greater_is_better=False,
)

In [3]:
trainer = SFTTrainer(
	model=model,
	tokenizer=tokenizer,
	train_dataset=train_dataset,
	eval_dataset=val_dataset,
	dataset_text_field="text",
	max_seq_length=MAX_SEQ_LENGTH,
	dataset_num_proc=12,
	packing=True,
	args=args,
	callbacks=[
		EarlyStoppingCallback(
			early_stopping_patience=3,
			early_stopping_threshold=0.005
		)
	],
)

Unsloth: Tokenizing ["text"] (num_proc=20): 100%|██████████| 100/100 [00:06<00:00, 16.53 examples/s]


In [4]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3090. Max memory = 23.663 GB.
1.131 GB of memory reserved.


In [5]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 13,042 | Num Epochs = 3 | Total steps = 102
O^O/ \_/ \    Batch size per device = 14 | Gradient accumulation steps = 28
\        /    Data Parallel GPUs = 1 | Total batch size (14 x 28 x 1) = 392
 "-____-"     Trainable parameters = 52,183,040 of 1,052,068,992 (4.96% trained)
Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


OutOfMemoryError: CUDA out of memory. Tried to allocate 14.00 GiB. GPU 0 has a total capacity of 23.66 GiB of which 13.78 GiB is free. Including non-PyTorch memory, this process has 9.36 GiB memory in use. Of the allocated memory 8.99 GiB is allocated by PyTorch, and 40.90 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)